<a href="https://colab.research.google.com/github/revelacion1dev/Stego_Lab/blob/main/Sponge_AlgorithmXOF_And_Number_As_Operation_XOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sponge based algorithms works

The idea of this colab is to understand the base of how sponge algorithms works to understand the way the output of length is variable. (This is the baseline of SHAKE 256 that I use in FIPS205 implementation)

In [ ]:
def rotate_left(bits, n, size=8):
    """Rota bits a la izquierda n posiciones en un entero de tamaño `size`."""
    return ((bits << n) | (bits >> (size - n))) & ((1 << size) - 1)

def sponge_hash(input_bits, r=4, c=4, output_blocks=3):
    # Paso 1: Padding (simplificado): añadir bloque '0001'
    blocks = [input_bits[i:i+r] for i in range(0, len(input_bits), r)]
    blocks.append('0001')  # padding block
    print(f"[+] Bloques con padding: {blocks}")

    # Paso 2: Inicializar estado de 8 bits
    state = 0b00000000
    print(f"[+] Estado inicial: {state:08b}")

    # Paso 3: Absorción
    for i, block in enumerate(blocks):
        block_int = int(block, 2)
        rate_mask = (1 << r) - 1
        state_rate = state & rate_mask
        new_rate = state_rate ^ block_int

        # Combinar con capacity (bits más altos no se tocan aquí)
        state = (state & (~rate_mask)) | new_rate
        print(f"[-] Absorbiendo bloque {i}: {block} → XOR con rate: {state_rate:04b} ^ {block_int:04b} = {new_rate:04b}")
        print(f"    Estado antes permutación: {state:08b}")

        # Permutación (rotación a la izquierda)
        state = rotate_left(state, 1)
        print(f"    Estado después permutación: {state:08b}")

    # Paso 4: Squeeze (exprimir salida)
    output = []
    for i in range(output_blocks):
        out_block = state & rate_mask
        output.append(f"{out_block:04b}")
        print(f"[#] Bloque de salida {i}: {out_block:04b}")
        state = rotate_left(state, 1)

    return output

# Simular con input de 8 bits
entrada_binaria = '10111100'
salida = sponge_hash(entrada_binaria)
print(f"\n🎯 Salida final del hash (en bloques): {salida}")
print(f"🔑 Salida concatenada: {''.join(salida)}")


[+] Bloques con padding: ['1011', '1100', '0001']
[+] Estado inicial: 00000000
[-] Absorbiendo bloque 0: 1011 → XOR con rate: 0000 ^ 1011 = 1011
    Estado antes permutación: 00001011
    Estado después permutación: 00010110
[-] Absorbiendo bloque 1: 1100 → XOR con rate: 0110 ^ 1100 = 1010
    Estado antes permutación: 00011010
    Estado después permutación: 00110100
[-] Absorbiendo bloque 2: 0001 → XOR con rate: 0100 ^ 0001 = 0101
    Estado antes permutación: 00110101
    Estado después permutación: 01101010
[#] Bloque de salida 0: 1010
[#] Bloque de salida 1: 0100
[#] Bloque de salida 2: 1001

🎯 Salida final del hash (en bloques): ['1010', '0100', '1001']
🔑 Salida concatenada: 101001001001


## Forzando una salida variable

In [ ]:
def rotate_left(x, n, size=16):
    """Rotación circular izquierda de x con n posiciones."""
    return ((x << n) | (x >> (size - n))) & ((1 << size) - 1)

def pad_input(bin_str, r):
    """Rellena el mensaje para que sea múltiplo de r y agrega fin de dominio."""
    blocks = [bin_str[i:i+r] for i in range(0, len(bin_str), r)]
    if len(blocks[-1]) < r:
        blocks[-1] = blocks[-1].ljust(r, '0')
    blocks.append('1'.ljust(r, '0'))  # Dominio padding (simplificado)
    return blocks

def sponge_hash_variable_output(message_bits, output_length_bits=256, r=8, c=8):
    assert r + c == 16, "Estado total debe ser de 16 bits"
    blocks = pad_input(message_bits, r)
    print(f"[+] Bloques de entrada (con padding): {blocks}")

    # Estado interno: 16 bits inicializado en 0
    state = 0b0000000000000000
    print(f"[+] Estado inicial: {state:016b}")

    # Absorción
    for i, block in enumerate(blocks):
        block_val = int(block, 2)
        state_rate = state & ((1 << r) - 1)
        new_rate = state_rate ^ block_val
        state = (state & (~((1 << r) - 1))) | new_rate
        print(f"[-] Bloque {i}: XOR {state_rate:08b} ^ {block_val:08b} = {new_rate:08b}")
        print(f"    Estado antes de permutar: {state:016b}")
        state = rotate_left(state, 3)
        print(f"    Estado después de permutar: {state:016b}")

    # Squeeze (extraer salida)
    output = ""
    total_blocks = output_length_bits // r
    for i in range(total_blocks):
        out_block = state & ((1 << r) - 1)
        output += f"{out_block:08b}"
        print(f"[#] Bloque de salida {i}: {out_block:08b}")
        state = rotate_left(state, 3)

    print(f"\n🔑 Digest final ({output_length_bits} bits): {output}")
    return output

# 🧪 Probar con mensaje binario arbitrario
mensaje_binario = '1011001110001111'  # 16 bits de entrada
digest = sponge_hash_variable_output(mensaje_binario, output_length_bits=256)


[+] Bloques de entrada (con padding): ['10110011', '10001111', '10000000']
[+] Estado inicial: 0000000000000000
[-] Bloque 0: XOR 00000000 ^ 10110011 = 10110011
    Estado antes de permutar: 0000000010110011
    Estado después de permutar: 0000010110011000
[-] Bloque 1: XOR 10011000 ^ 10001111 = 00010111
    Estado antes de permutar: 0000010100010111
    Estado después de permutar: 0010100010111000
[-] Bloque 2: XOR 10111000 ^ 10000000 = 00111000
    Estado antes de permutar: 0010100000111000
    Estado después de permutar: 0100000111000001
[#] Bloque de salida 0: 11000001
[#] Bloque de salida 1: 00001010
[#] Bloque de salida 2: 01010000
[#] Bloque de salida 3: 10000011
[#] Bloque de salida 4: 00011100
[#] Bloque de salida 5: 11100000
[#] Bloque de salida 6: 00000101
[#] Bloque de salida 7: 00101000
[#] Bloque de salida 8: 01000001
[#] Bloque de salida 9: 00001110
[#] Bloque de salida 10: 01110000
[#] Bloque de salida 11: 10000010
[#] Bloque de salida 12: 00010100
[#] Bloque de salida 

## Number As Operation: Create an Extandable Hash function based on Cesar concept and defining number as operations

  The idea here is simple. Is to base on the cesar table use the numbeer as operations and create some dividers so I know the transition to the next number.

  The goal here is to :  

    * Create a different lenght information package so decrypytion process is

    * Start for simplicity then go complex

    * Destroy the entropy of the first message so not pattern recognizer is able to follow

## Cesar XOF : First implementation view


  This is a first implementation of the concept

In [15]:
import random

class SimpleCaesarXOF:
    """
    Simple Caesar XOF that actually works.
    """

    def __init__(self):
        # Operations
        self.transformations = {
            1: +2, 2: +4, 3: -2, 4: +8, 5: -1,
            6: +3, 7: -4, 8: +1, 9: -3
            # 0 is DIVIDER
        }

    def generate_sequence_for_char(self, char):
        """
        Generate simple sequence that reaches UTF-8 value.
        Random choice: exact or with small entropy.
        """
        target = ord(char)
        sequence = []
        accumulator = 0

        # Random: 70% exact, 30% small entropy
        add_entropy = random.random() < 0.3

        # Build sequence to reach target
        while accumulator < target:
            # If close to target, be more precise
            if target - accumulator <= 10:
                # Find best exact operation
                best_op = 8  # +1 default
                for op_num in [8, 6, 2, 1]:  # Small positive operations first
                    if accumulator + self.transformations[op_num] <= target:
                        best_op = op_num
                        break
                sequence.append(best_op)
                accumulator += self.transformations[best_op]
            else:
                # Can use bigger operations
                if add_entropy and random.random() < 0.5:
                    # Random operation
                    op_num = random.choice([1, 2, 4, 6, 8])  # Only positive
                else:
                    # Best fitting operation
                    op_num = 4 if target - accumulator >= 8 else 2  # +8 or +4

                sequence.append(op_num)
                accumulator += self.transformations[op_num]

        return sequence, accumulator

    def encrypt_message(self, message):
        """Encrypt message."""
        encrypted_sequence = []
        details = []

        for char in message:
            sequence, final_value = self.generate_sequence_for_char(char)
            details.append((char, sequence, final_value))

            # Add sequence + divider
            encrypted_sequence.extend(sequence)
            encrypted_sequence.append(0)

        # Convert to string without spaces
        encrypted_code = ''.join(map(str, encrypted_sequence))

        return encrypted_code, details

    def decrypt_message(self, encrypted_code):
        """Decrypt code."""
        # Convert string to list of numbers
        try:
            numbers = [int(c) for c in encrypted_code]
        except ValueError:
            return "ERROR"

        decrypted_chars = []
        current_sequence = []

        for num in numbers:
            if num == 0:  # Divider
                if current_sequence:
                    # Apply operations
                    accumulator = 0
                    for op_num in current_sequence:
                        if op_num in self.transformations:
                            accumulator += self.transformations[op_num]

                    # Convert to character
                    try:
                        char = chr(accumulator)
                        decrypted_chars.append(char)
                    except ValueError:
                        decrypted_chars.append('?')

                    current_sequence = []
            else:
                current_sequence.append(num)

        return ''.join(decrypted_chars)

    def show_dividers(self, encrypted_code):
        """Show dividers clearly."""
        result = ""
        current = ""

        for c in encrypted_code:
            if c == '0':
                if current:
                    result += f"[{current}]"
                    current = ""
                result += "|"
            else:
                current += c

        if current:
            result += f"[{current}]"

        return result

def main():
    """Simple working interface."""

    print("🎯 CAESAR XOF SIMPLE")
    print("=" * 30)
    print()

    xof = SimpleCaesarXOF()

    while True:
        message = input("Mensaje: ").strip()

        if not message:
            print("¡Adiós! 👋")
            break

        # Encrypt
        encrypted_code, details = xof.encrypt_message(message)

        # Decrypt to verify
        verification = xof.decrypt_message(encrypted_code)

        # Show dividers
        divider_view = xof.show_dividers(encrypted_code)

        # Results
        print(f"\nMensaje: '{message}'")
        print(f"Código: {encrypted_code}")
        print(f"Dividers: {divider_view}")
        print(f"Verificación: '{verification}'")

        # Show details
        print(f"\nDetalles:")
        for char, sequence, final_val in details:
            utf8_val = ord(char)
            print(f"  '{char}' (UTF-8:{utf8_val}) → {sequence} → {final_val}")

        # Status
        if verification == message:
            print("\n✅ CORRECTO")
        else:
            print("\n❌ ERROR")

        print("\n" + "="*30)
        print()

if __name__ == "__main__":
    main()

🎯 CAESAR XOF SIMPLE

Mensaje: hola

Mensaje: 'hola'
Código: 444444444444888888880214464484448444484888068624464444164414488888880444444444448888888880
Dividers: [44444444444488888888]|[214464484448444484888]|[6862446444416441448888888]|[44444444444888888888]|
Verificación: 'hola'

Detalles:
  'h' (UTF-8:104) → [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8] → 104
  'o' (UTF-8:111) → [2, 1, 4, 4, 6, 4, 4, 8, 4, 4, 4, 8, 4, 4, 4, 4, 8, 4, 8, 8, 8] → 111
  'l' (UTF-8:108) → [6, 8, 6, 2, 4, 4, 6, 4, 4, 4, 4, 1, 6, 4, 4, 1, 4, 4, 8, 8, 8, 8, 8, 8, 8] → 108
  'a' (UTF-8:97) → [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8] → 97

✅ CORRECTO


Mensaje: hola que tal me gustaria ver que tan largo puedo extender esto

Mensaje: 'hola que tal me gustaria ver que tan largo puedo extender esto'
Código: 4444444444448888888804444444444444888888804444444444444888804444444444488888888804448888888804444444444444888888888044444444444444888880444444444444888880444888888880444444444

KeyboardInterrupt: Interrupted by user